In [1]:
from vedo import Points, Plotter, Line, Axes, Plane, Grid, Sphere, Text2D, shapes
import sys
from IPython.display import Video
import imageio.v2 as imageio
import os
import shutil
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.io import savemat, loadmat
import scipy.io as spio

In [2]:
def generate_video_from_raw_data(file_name, raw_data_folder, prints = False):

    if not prints:
        print("Generating video...")
        print("May take a while: use prints = True to see progress...")
    
    # Load data
    marker_data = pd.read_table(raw_data_folder+file_name+".tsv", 
                        sep = '\t',
                        skiprows = range(0, 11)
                        )
    
    marker_data = marker_data.drop(marker_data.columns[marker_data.columns.str.contains('unnamed', case=False)], axis=1)
    
    marker_start_time = pd.read_table(raw_data_folder+file_name+".tsv",
                                      skiprows = [*range(0, 7), 8], 
                                      header=None,
                                      on_bad_lines = 'skip').to_numpy()

    print(marker_start_time)
    
    marker_start_time = datetime.strptime(marker_start_time[:, 1][0], '%Y-%m-%d, %H:%M:%S.%f').timestamp() # add an hour for daylight savings and some adjustment
    
    marker_data_np = marker_data.drop(["Time", "Frame"], axis=1).to_numpy()
    
    reshaped = marker_data_np.reshape(marker_data_np.shape[0], -1, 3)
    reshaped[:, :, [1, 2]] = reshaped[:, :, [2, 1]]
    marker_data_np = reshaped.reshape(marker_data_np.shape)

    if prints:
        print(f"Raw data shape: {marker_data_np.shape}")
        print()
    
    # Calculate number of points per frame (each point has 3 coords: x,y,z)
    num_points = marker_data_np.shape[1] // 3
    
    # If folder exists, delete it completely
    if os.path.exists("output"):
        shutil.rmtree("output")
    
    # Create empty folder
    os.makedirs("output")
    
    # Initialize Vedo Plotter
    vp = Plotter(offscreen=True, bg='white', size=(800, 608))
    
    # Set camera view
    vp.camera.SetPosition(8, 8, 8)
    vp.camera.SetFocalPoint(0, 0, 0)
    vp.camera.SetViewUp(0, 1, 0)
    vp.render()
    
    grid = Grid(pos=(0,0,0), s=(100,100), res=(200,200), c='lightgray', alpha=0.3)
    grid.rotate(angle=90, axis=(1,0,0))  # rotate 90 degrees around X axis
    vp.add(grid)
    
    # format column names for selection
    point_names = marker_data.columns.to_list()
    point_names = [x for x in point_names if x[-1] not in ["Y", "Z"]]
    point_names.remove("Time")
    point_names.remove("Frame")
    point_names = [x[:-2] for x in point_names]

    connections = [
        # hip
        (point_names.index('BACK_REF'), point_names.index('BACK_X')),
        (point_names.index('BACK_X'), point_names.index('BACK_O')),
        (point_names.index('BACK_O'), point_names.index('BACK_Y')),
        #right foot
        (point_names.index('R_TOE'), point_names.index('R_MET5')),
        (point_names.index('R_TOE'), point_names.index('R_HEEL')),
        (point_names.index('R_TOE'), point_names.index('R_REF')),
        (point_names.index('R_REF'), point_names.index('R_MET5')),
        (point_names.index('R_REF'), point_names.index('R_HEEL')),
        (point_names.index('R_MET5'), point_names.index('R_HEEL')),
        #left foot
        (point_names.index('L_TOE'), point_names.index('L_MET5')),
        (point_names.index('L_TOE'), point_names.index('L_HEEL')),
        (point_names.index('L_TOE'), point_names.index('L_REF')),
        (point_names.index('L_REF'), point_names.index('L_MET5')),
        (point_names.index('L_REF'), point_names.index('L_HEEL')),
        (point_names.index('L_MET5'), point_names.index('L_HEEL')),
        #wrist    
        (point_names.index('WRIST'), point_names.index('WRIST_LAT')),
        (point_names.index('WRIST_MED'), point_names.index('WRIST_LAT')),
    ]

    # list to hold things to be deleted
    dynamic_actors = []
    
    # Animation loop
    for i, frame in enumerate(marker_data_np):
        pts_coords = frame.reshape((num_points, 3)) / 1000 # convert from mm to m

        if prints:
            print(f"Frame {i}, points shape: {pts_coords.shape}, first point: {pts_coords[0]}")
    
        # Remove previous dynamic actors
        for actor in dynamic_actors:
            vp.remove(actor)
        dynamic_actors.clear()
    
        # Create points and lines
        pts = Points(pts_coords, r=3, c='blue')
    
        for idx1, idx2 in connections:
            p1 = pts_coords[idx1]
            p2 = pts_coords[idx2]
        
            if not(np.allclose(p1, 0)) and not(np.allclose(p2, 0)):
                line = Line(p1, p2, c='black', lw=2)
                vp.add(line)
                dynamic_actors.append(line)
            else:
                # Optionally: log skipped lines
                if prints:
                    print(f"Skipping line between {idx1} and {idx2} due to missing data")
    
        # Add to scene
        vp.add(pts, line)
        dynamic_actors.extend([pts, line])
    
        # Render and save frame
        vp.render()
        vp.screenshot(f"output/frame_{i:04d}.png")
    
    # Finalize
    vp.close()

    # Create video from saved frames
    image_folder = "output"
    fps = 100  # frames per second
    
    images = sorted([img for img in os.listdir(image_folder) if img.endswith(".png")])
    with imageio.get_writer(raw_data_folder+file_name+".mp4", fps=fps) as writer:
        for filename in images:
            image_path = os.path.join(image_folder, filename)
            image = imageio.imread(image_path)
            writer.append_data(image)

    print(f"Video saved to {raw_data_folder+file_name+".mp4"}")
    print()
    
    # get rid of outputs folder
    if os.path.exists("output"):
        shutil.rmtree("output")

In [3]:
raw_data_folder = r"C:\Users\ac4jmi\Desktop\DMO4LNC\Data Collection\Dataset\CP\718\Lab\Motion Capture Data\\"

tsv_files = [
    os.path.splitext(f)[0]
    for f in os.listdir(raw_data_folder)
    if f.endswith('.tsv') and not f.startswith('synced')
]

for file_name in tsv_files:
    generate_video_from_raw_data(file_name, raw_data_folder, prints = False)
    Video(raw_data_folder+file_name+".mp4", embed=True, width=608)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\ac4jmi\\Desktop\\DMO4LNC\\Data Collection\\Dataset\\CP\\718\\Lab\\Motion Capture Data\\\\'

In [33]:
# loadmat from scipy.io imports as a numpy array beyond the first layer, this modified version imports it as an actual dictionary
# taken from stack overflow
def loadmat_fixed(filename):
    data = loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def _check_keys(d):
    for key in d:
        d[key] = _check_element(d[key])
    return d

def _check_element(elem):
    if isinstance(elem, spio.matlab.mio5_params.mat_struct):
        return _todict(elem)
    elif isinstance(elem, np.ndarray):
        return _tolist(elem)
    else:
        return elem

def _todict(matobj):
    d = {}
    for fieldname in matobj._fieldnames:
        elem = matobj.__dict__[fieldname]
        d[fieldname] = _check_element(elem)
    return d

def _tolist(ndarray):
    lst = []
    for sub_elem in ndarray:
        lst.append(_check_element(sub_elem))
    return lst

def get_plottable_data(data_mat):
    # get markers
    markers = pd.DataFrame()
    for marker in data_mat["Standards"]["Stereophoto_raw"]["Mrks"].keys():
        temp_df = pd.DataFrame(data = data_mat["Standards"]["Stereophoto_raw"]["Mrks"][marker], columns = [marker+"_X", marker+"_Y", marker+"_Z"])
        markers = pd.concat([markers, temp_df], axis=1)

    # get imu data
    waist_imu_data = pd.DataFrame(data_mat["SU"]["LowerBack"]["Acc"], columns = ["Acc_X", "Acc_Y", "Acc_Z"])
    
    # get ic and fc events
    # check if we have more than 1 walking bouts
    if len(data_mat["Standards"]["Stereophoto"]["ContinuousWalkingPeriod"]) > 60:
        ics = data_mat["Standards"]["Stereophoto"]["ContinuousWalkingPeriod"]["InitialContact_Event"]
    else:
        ics = []
        for cwp in data_mat["Standards"]["Stereophoto"]["ContinuousWalkingPeriod"]:
            ics.extend(cwp["InitialContact_Event"])

    
    # get time and normalise
    t_0 = data_mat["SU"]["LowerBack"]["Timestamp"][0]
    timestamp = pd.DataFrame(data_mat["SU"]["LowerBack"]["Timestamp"] - t_0, columns=["time"])
    
    ic_indices = []
    
    for ic in ics:
        # Find the index of the timestamp closest to the event
        idx = (timestamp["time"] - ic).abs().idxmin()
        ic_indices.append(idx)
    
    return markers, waist_imu_data, timestamp, ic_indices

def generate_mobgap_mocap_videos(data_mat_path):
    data_mat_path_full = os.path.join(data_mat_path, "data.mat")
    print(f"Loading data from {data_mat_path_full}")
    data_mat = loadmat_fixed(data_mat_path_full)
    for test in data_mat["data"]["TimeMeasure1"].keys():
        # skip standing
        if test == "Test1":
            continue
        for trial in data_mat["data"]["TimeMeasure1"][test].keys():
            marker_data, waist_imu_data, timestamps, ic_indices = get_plottable_data(data_mat["data"]["TimeMeasure1"][test][trial])
            
            filename = f"{test}_{trial}.mp4"
            save_full_path = os.path.join(data_mat_path, filename)
            
            vis = MotionVisualizer(marker_data, waist_imu_data, timestamps, ic_indices)
            
            # Pass the path to start()
            vis.start(save_path=save_full_path)
            
            # To save video (requires modifying run_animation as mentioned above):
            # vis.start(save_path=f"{test}_{trial}.mp4")
            
            #print(markers)
            #print(waist_imu_data)

In [34]:
import pandas as pd
import numpy as np
import vedo
from vedo import Plotter, Sphere, Lines, Video, Grid

# Force VTK backend for Jupyter
vedo.settings.default_backend = 'vtk' 

class MotionVisualizer:
    def __init__(self, markers_df, imu_df, time_df, ic_indices):
        self.markers_df = markers_df
        self.imu_df = imu_df
        self.time_df = time_df
        self.ic_indices = ic_indices
        
        # 1. Parse Data & Units
        self.times = time_df["time"].values
        raw_m = markers_df.values
        n_frames = raw_m.shape[0]
        n_cols = raw_m.shape[1]
        
        self.marker_pos = raw_m.reshape(n_frames, n_cols // 3, 3)
        
        # FIX: Swap Y/Z (Y=Up) and convert mm -> m
        self.marker_pos[:, :, [1, 2]] = self.marker_pos[:, :, [2, 1]]
        self.marker_pos = self.marker_pos / 1000.0
        self.n_frames = n_frames
        
        # 2. Extract Marker Names and Indices
        # We assume columns are "NAME_X", "NAME_Y", "NAME_Z"
        # So we take every 3rd column and strip the last 2 chars ("_X")
        all_cols = markers_df.columns.tolist()
        marker_names = [c[:-2] for c in all_cols[0::3]]
        
        # --- UPDATED CONNECTIONS TO MATCH YOUR DATAFRAME ---
        target_connections = [
            # Hip / Back (Updated names: BACKREF, BACKX, BACK0, BACKY)
            ('BACKREF', 'BACKX'), 
            ('BACKX', 'BACK0'), 
            ('BACK0', 'BACKY'),
            
            # Right Foot (Updated names: RTOE, RMET5, RHEEL, RREF)
            ('RTOE', 'RMET5'), 
            ('RTOE', 'RHEEL'), 
            ('RTOE', 'RREF'),
            ('RREF', 'RMET5'), 
            ('RREF', 'RHEEL'), 
            ('RMET5', 'RHEEL'),
            
            # Left Foot (Assuming similar naming: LTOE, LMET5, LHEEL, LREF)
            ('LTOE', 'LMET5'), 
            ('LTOE', 'LHEEL'), 
            ('LTOE', 'LREF'),
            ('LREF', 'LMET5'), 
            ('LREF', 'LHEEL'), 
            ('LMET5', 'LHEEL'),
            
            # Wrist (Assuming WRIST, WRISTLAT, WRISTMED based on typical sets)
            ('WRIST', 'WRISTLAT'), 
            ('WRISTMED', 'WRISTLAT')
        ]
        
        self.conn_indices = []
        for name1, name2 in target_connections:
            try:
                idx1 = marker_names.index(name1)
                idx2 = marker_names.index(name2)
                self.conn_indices.append((idx1, idx2))
            except ValueError:
                # print(f"Warning: Connection {name1}-{name2} skipped (markers not found)")
                pass

        # 3. Setup Plotter
        self.plt = Plotter(shape=(2, 1), sharecam=False, size=(800, 1000), offscreen=True, bg='white')

    def start(self, save_path="output.mp4"):
        if save_path is None: save_path = "output.mp4"
        print(f"Setting up scenes for: {save_path}")
        
        # --- TOP VIEWPORT: MOCAP ---
        self.plt.at(0)
        
        # Floor Grid
        grid = Grid(pos=(0,0,0), s=(40, 40), res=(40, 40), c='lightgray', alpha=0.3)
        grid.rotate(angle=90, axis=(1, 0, 0))
        self.plt += grid
        
        # Initial Points
        initial_coords = self.marker_pos[0]
        self.mocap_points = []
        
        for i in range(len(initial_coords)):
            s = Sphere(pos=initial_coords[i], r=0.015, c="blue") 
            self.mocap_points.append(s)
            self.plt += s
            
        self.skeleton_lines = None 
        
        # Initialize Camera (Position will be updated in loop)
        cam0 = self.plt.at(0).camera
        cam0.SetViewUp(0, 1, 0)
        cam0.SetPosition(8, 8, 8)
        cam0.SetFocalPoint(0, 0, 0)

        # --- BOTTOM VIEWPORT: IMU ---
        self.plt.at(1)
        self.plt += vedo.Text2D("IMU Acceleration", pos="top-left", c="black")
        
        z_zeros = np.zeros_like(self.times)
        pts_x = np.column_stack((self.times, self.imu_df["Acc_X"].values, z_zeros))
        pts_y = np.column_stack((self.times, self.imu_df["Acc_Y"].values, z_zeros))
        pts_z = np.column_stack((self.times, self.imu_df["Acc_Z"].values, z_zeros))
        
        self.plt += [
            vedo.Line(pts_x, c="red", lw=2),
            vedo.Line(pts_y, c="green", lw=2),
            vedo.Line(pts_z, c="blue", lw=2)
        ]
        
        y_min = self.imu_df.min().min()
        y_max = self.imu_df.max().max()
        
        for idx in self.ic_indices:
            if idx < len(self.times):
                t = self.times[idx]
                self.plt += vedo.Line([(t, y_min, 0), (t, y_max, 0)], c="black", lw=2, alpha=0.5)

        self.cursor = vedo.Line([(0, y_min, 0), (0, y_max, 0)], c="gold", lw=4)
        self.plt += self.cursor

        cam1 = self.plt.at(1).camera
        cam1.SetParallelProjection(True)
        
        self.run_animation(save_path)

    def run_animation(self, save_path):
        y_center = (self.imu_df.max().max() + self.imu_df.min().min()) / 2
        y_range = self.imu_df.max().max() - self.imu_df.min().min()
        
        video = Video(save_path, backend='cv2', fps=50)
        step = 2
        
        print(f"Processing {self.n_frames} frames...")
        
        # Camera tracking offset (X, Y, Z) meters relative to subject
        camera_offset = np.array([5.0, 3.0, 5.0]) 
        
        for idx in range(0, self.n_frames, step): 
            if idx % 50 == 0: print(f"Frame {idx}/{self.n_frames}", end='\r')

            # --- UPDATE MOCAP ---
            current_pos = self.marker_pos[idx]
            
            # 1. Update Points & Calculate Center
            valid_points = []
            for i, actor in enumerate(self.mocap_points):
                p = current_pos[i]
                actor.pos(p)
                if not np.allclose(p, 0):
                    valid_points.append(p)
            
            # 2. Update Lines
            valid_starts, valid_ends = [], []
            for i1, i2 in self.conn_indices:
                p1, p2 = current_pos[i1], current_pos[i2]
                if not (np.allclose(p1, 0) or np.allclose(p2, 0)):
                    valid_starts.append(p1)
                    valid_ends.append(p2)
            
            if self.skeleton_lines: self.plt.at(0).remove(self.skeleton_lines)
            if valid_starts:
                self.skeleton_lines = Lines(valid_starts, valid_ends, c='black', lw=2)
                self.plt.at(0).add(self.skeleton_lines)

            # 3. Update Camera Tracking
            if valid_points:
                center_of_mass = np.mean(valid_points, axis=0)
                cam0 = self.plt.at(0).camera
                cam0.SetFocalPoint(center_of_mass)
                cam0.SetPosition(center_of_mass + camera_offset)
                cam0.SetViewUp(0, 1, 0)

            # --- UPDATE IMU ---
            t = self.times[idx]
            self.cursor.x(t)
            cam = self.plt.at(1).camera
            cam.SetFocalPoint(t, y_center, 0)
            cam.SetPosition(t, y_center, 10)
            cam.SetParallelScale(y_range * 0.6) 

            self.plt.render()
            video.add_frame()

        video.close()
        self.plt.close()
        print(f"\nSaved: {save_path}")

In [35]:
data_mat_folder = r"C:\Users\ac4jmi\Desktop\DMO4LNC\dmo4lnc-analysis\Dataset\CP\383\Lab\Laboratory"
generate_mobgap_mocap_videos(data_mat_folder)

Loading data from C:\Users\ac4jmi\Desktop\DMO4LNC\dmo4lnc-analysis\Dataset\CP\383\Lab\Laboratory\data.mat


C:\Users\ac4jmi\AppData\Local\Temp\ipykernel_39384\1210194949.py:13: DeprecationWarning: Please import `mat_struct` from the `scipy.io.matlab` namespace; the `scipy.io.matlab.mio5_params` namespace is deprecated and will be removed in SciPy 2.0.0.
  if isinstance(elem, spio.matlab.mio5_params.mat_struct):


Setting up scenes for: C:\Users\ac4jmi\Desktop\DMO4LNC\dmo4lnc-analysis\Dataset\CP\383\Lab\Laboratory\Test2_Trial1.mp4
Processing 1647 frames...:\Users\ac4jmi\Desktop\DMO4LNC\dmo4lnc-analysis\Dataset\CP\383\Lab\Laboratory\Test2_Trial1.mp4 is open... 
Frame 1600/1647
Saved: C:\Users\ac4jmi\Desktop\DMO4LNC\dmo4lnc-analysis\Dataset\CP\383\Lab\Laboratory\Test2_Trial1.mp4
Setting up scenes for: C:\Users\ac4jmi\Desktop\DMO4LNC\dmo4lnc-analysis\Dataset\CP\383\Lab\Laboratory\Test3_Trial1.mp4
Processing 1933 frames...:\Users\ac4jmi\Desktop\DMO4LNC\dmo4lnc-analysis\Dataset\CP\383\Lab\Laboratory\Test3_Trial1.mp4 is open... 
Frame 1900/1933
Saved: C:\Users\ac4jmi\Desktop\DMO4LNC\dmo4lnc-analysis\Dataset\CP\383\Lab\Laboratory\Test3_Trial1.mp4
Setting up scenes for: C:\Users\ac4jmi\Desktop\DMO4LNC\dmo4lnc-analysis\Dataset\CP\383\Lab\Laboratory\Test4_Trial1.mp4
Processing 4296 frames...:\Users\ac4jmi\Desktop\DMO4LNC\dmo4lnc-analysis\Dataset\CP\383\Lab\Laboratory\Test4_Trial1.mp4 is open... 
Frame 425